In [44]:
import pandas as pd
import ipaddress
import random
import numpy as np

In [2]:
raw_csv_df = pd.read_csv("events.csv")

In [3]:
raw_csv_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


In [8]:
ip_db = pd.read_csv("IP2LOCATION-LITE-DB1.CSV", names=["ip_from", "ip_to", "country_code", "country_name"])

In [9]:
ip_db.head()

,ip_from,ip_to,country_code,country_name
0,0,16777215,-,-
1,16777216,16777471,AU,Australia
2,16777472,16778239,CN,China
3,16778240,16779263,AU,Australia
4,16779264,16781311,CN,China


In [18]:
ipa = ipaddress.IPv4Address(16777216)
ipa

IPv4Address('1.0.0.0')

In [17]:
int(ipa)

16777216

In [20]:
countries_to_select_ip = ["Bolivia (Plurinational State of)", "Colombia", "Japan", "Italy", "Ireland"]

In [21]:
ip_db[ip_db["country_name"] == countries_to_select_ip[0]]

,ip_from,ip_to,country_code,country_name
4590,210784768,210785023,BO,Bolivia (Plurinational State of)
4592,210785792,210786303,BO,Bolivia (Plurinational State of)
21048,576979968,576980991,BO,Bolivia (Plurinational State of)
21057,576990720,576991231,BO,Bolivia (Plurinational State of)
26223,653670912,653671423,BO,Bolivia (Plurinational State of)
...,...,...,...,...
239352,3451508224,3451508479,BO,Bolivia (Plurinational State of)
240081,3455132160,3455133695,BO,Bolivia (Plurinational State of)
240910,3463156736,3463157759,BO,Bolivia (Plurinational State of)
242871,3489775104,3489775359,BO,Bolivia (Plurinational State of)


In [57]:
ip_ranges_for_countries = ip_db[ip_db["country_name"].isin(countries_to_select_ip)]

def generate_random_ip() -> str:
    random_ip_range = ip_ranges_for_countries.iloc[random.randint(0, len(ip_ranges_for_countries)-1)]
    random_ip = int(random_ip_range["ip_from"]) + random.randint(0,255)
    return ipaddress.IPv4Address(random_ip).compressed
random_ip_per_user = {user_id: generate_random_ip() for user_id in raw_csv_df["user_id"].unique()}

In [59]:
events_with_ip_df  = raw_csv_df.copy()
events_with_ip_df["ip"] = raw_csv_df["user_id"].map(lambda user_id: random_ip_per_user[user_id])

In [61]:
raw_csv_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


In [60]:
events_with_ip_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,ip
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT,103.179.45.110
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY,149.19.44.164
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy,203.137.144.104
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08,23.61.196.56
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ,87.120.34.208


In [63]:
events_with_ip_df.to_csv("events_with_ip.csv", index=False)